# Advanced Querying Mongo

Importing libraries and setting up connection

In [73]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [74]:
collection = client.get_database('companies').get_collection('companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [36]:
list(collection.find({'name':'Babelgum'},{'name':1, '_id':0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [38]:
big_companies =list(collection.find({'number_of_employees':{'$gt':5000}},{'name':1, 'number_of_employees':1, '_id':0}).limit(20).sort([('number_of_employees',-1)]))

In [39]:
big_companies[0:5]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [43]:
comp_year = list(collection.find({'founded_year': {'$gte':2000, '$lte': 2005}}, {'name':1, 'founded_year':1, '_id':0}
))
comp_year[0:5]

[{'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Digg', 'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [45]:
val_and_found = list(collection.find({'ipo.valuation_amount': {'$gt':100000000}, 'founded_year':{'$lt': 2010}},{'name':1, 'ipo':1, '_id':0}))
val_and_found[0:3]

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [46]:
small_comp_found = list(collection.find({'number_of_employees': {'$lt':1000}, 'founded_year':{'$lt': 2005}}, {'name':1, 'number_of_employees':1,'founded_year':1, '_id':0}).sort([('number_of_employees',-1)]).limit(10))
small_comp_found[0:5]

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999}]

### 6. All the companies that don't include the `partners` field.

In [49]:
partners_out = list(collection.find({'partners':{'$exists': False}}, {'name':1, '_id':0}))
partners_out[0:5]

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [50]:
null_on_category_field = list(collection.find({'category_code': { '$type': 10 }}, {'name':1, 'category_code':1, '_id':0}))
null_on_category_field[0:5]

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [51]:
hundred_to_thousand_empl = list(collection.find({'number_of_employees': { '$gte': 100 , '$lt':1000}}, {'name':1, 'number_of_employees':1, '_id':0}))
hundred_to_thousand_empl[0:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [53]:
IPO = list(collection.find({}, {'name':1, 'ipo':1, '_id':0}).sort([('ipo',-1)]))
IPO[0:3]

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [54]:
numb_of_empl = list(collection.find({}, {'name':1, 'number_of_employees':1, '_id':0}).sort([('number_of_employees',-1)]).limit(10))
numb_of_empl[0:5]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [55]:
second_sem_companies = list(collection.find({'founded_month': {'$gt':6}}, {'name':1, 'founded_month':1, '_id':0}).limit(1000))
second_sem_companies[0:5]

[{'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Joost', 'founded_month': 10}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [56]:
comp_found_bf_2000 = list(collection.find({'founded_year': {'$lt':2000}, 'acquisition.price_amount': {'$gt':10000}},  {'name':1, 'founded_year':1,'acquisition.price_amount':1, '_id':0}))
comp_found_bf_2000[0:5]

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [59]:
acq_after_2010 = list(collection.find({'acquisition.acquired_year': {'$gt':2010}}, {'name':1, 'acquisition':1, '_id':0}).sort([('acquisition.price_amount', 1)]))
acq_after_2010[0:2]

[{'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [60]:
found_year = list(collection.find({},{'name':1, 'founded_year':1,'_id':0}).sort([('founded_year',-1)]))
found_year[0:5]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [61]:
found_first_seven_days = list(collection.find({'founded_day': {'$lte':7}}, {'name':1, 'founded_day':1,'acquisition.price_amount':1, '_id':0}).sort([('acquisition.price_amount',-1)]))
found_first_seven_days[0:5]

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [66]:
web_cat = list(collection.find({'category_code': 'web', 'number_of_employees': {'$gt':4000}}, {'name':1, 'category_code':1,'number_of_employees':1, '_id':0}).sort([('number_of_employees',1)]))
web_cat[0:5]

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [67]:
big_acq_amount = list(collection.find({'acquisition.price_currency_code': 'EUR', 'acquisition.price_amount': {'$gt':10000000}}, {'name':1, 'acquisition.price_currency_code':1,'acquisition.price_amount':1, '_id':0}))

big_acq_amount[0:5]

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [70]:
acq_first_sem = list(collection.find({'acquisition.acquired_month': {'$lte':3}}, {'name':1, 'acquisition':1, '_id':0}).limit(10))
acq_first_sem[0:2]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [71]:
two_thousand_and_ten = list(collection.find({'founded_year': {'$gte':2000, '$lte': 2010},'acquisition.acquired_year': {'$gte':2011}}, {'name':1, 'founded_year':1,'acquisition.acquired_year':1, '_id':0}))
two_thousand_and_ten[0:5]

[{'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2011}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [72]:
deadpooled = list(collection.aggregate([{'$project':{'name':1,'_id':0,'Diff_year': { '$subtract': ['$deadpooled_year', '$founded_year']}}}, { "$match" : {"Diff_year" : { "$gte" : 3 } } }
]))
deadpooled[0:5]

[{'name': 'Omnidrive', 'Diff_year': 3},
 {'name': 'Babelgum', 'Diff_year': 6},
 {'name': 'Thoof', 'Diff_year': 7},
 {'name': 'Wesabe', 'Diff_year': 5},
 {'name': 'Jangl SMS', 'Diff_year': 3}]